
# Preprocessing for MIT data(Binary classification) with MinMax Scaler and Oversamling
The default preprocessing step is what we conclude in our report. 
You can check below what should be input for this notebook and what would be the output if necessary.


Processing **steps** for MIT dataset:   
    resample: Oversampling  
    rescaling: MinMaxScaler  


**Input** : Cleaned data with dummy target variable.   
mitbih_test.csv   
mitbih_train.csv

**Output** : Sampled and Scaled data with dummy target:   
mitbih_train_clean_minmax_oversampling.csv  
mitbih_test_clean_minmax_oversampling.csv


In [1]:
import sys
import os
data_path = ''
data_output_path = ''
# Check if the environment is Google Colab
if 'google.colab' in sys.modules:
    print("Running on Google Colab")
    # Install required libraries
    !pip install scikit-learn -q
    !pip install pandas -q
    !pip install numpy -q
    !pip install imbalanced-learn -q


    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    # set the path where the csv file stored in your google drive. 
    data_path = '/content/drive/MyDrive/Heartbeat_Project/'
    data_output_path = data_path
    
else:
    print("Running on local environment")

    current_path = os.getcwd()
    print("Current working directory:", current_path)
    data_path = '../data/raw/'
    data_output_path = '../data/processed/'


Running on local environment
Current working directory: g:\Meine Ablage\heartbeat-analysis-ai\notebooks


In [2]:
# Verify installation and import libraries
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import RandomOverSampler


RawFiles = dict({
    'test': data_path +  'mitbih_test.csv', # MIT data set in which we have dummy-coded the target variable
    'train': data_path +  'mitbih_train.csv', # MIT data set in which we have dummy-coded the target variable
})

OutputFiles = dict({
    'test': data_output_path + 'mitbih_test_clean_minmax_oversampling.csv',
    'train': data_output_path + 'mitbih_train_clean_minmax_oversampling.csv',
})

In [3]:
def addColumnsToDataframe(df):
    """
    As the dataset is composed with 188 columns with the 188th columns as the category values,
    so we give the last column the name 'target', others named with 'c_182'
    """
    num_columns= df.shape[1]
    feature_col_name = ['c_' + str(i) for i in range(0, num_columns - 1)]
    df_columns = feature_col_name + ['target']
    df.columns = df_columns
    return df
def convertColumnAsInt(df, column):
    df[column] = pd.to_numeric(df[column], errors='coerce') # convert to numeric to handle NaN values
    df.dropna(subset=[column], inplace=True)  # drop the rows with NaN values
    df[column] = df[column].astype(int)  # convert to int
    return df

In [4]:
mitbih_train = pd.read_csv(RawFiles.get('train'), header=None ) 
mitbih_test = pd.read_csv(RawFiles.get('test'), header=None )

mitbih_train = addColumnsToDataframe(mitbih_train)
mitbih_train = convertColumnAsInt(mitbih_train, 'target')

mitbih_test = addColumnsToDataframe(mitbih_test)
mitbih_test = convertColumnAsInt(mitbih_test, 'target')

# target value and meanings
all_class_mapping = {
    0: 'Normal',
    1: 'Supraventricular',
    2: 'Ventricular',
    3: 'Fusion',
    4: 'Unclassifiable'
}
mitbih_train['target'] = mitbih_train['target'].map(all_class_mapping)
mitbih_test['target'] = mitbih_test['target'].map(all_class_mapping)
# Drop rows where 'target' is 'Unclassifiable beat'
mitbih_train = mitbih_train[mitbih_train['target'] != 'Unclassifiable']
mitbih_test = mitbih_test[mitbih_test['target'] != 'Unclassifiable']


# convert to binary classification Combine abnormal categories
mitbih_train['target'] = mitbih_train['target'].replace(['Supraventricular', 'Ventricular', 'Fusion'], 'abnormal')
mitbih_test['target'] = mitbih_test['target'].replace(['Supraventricular', 'Ventricular', 'Fusion'], 'abnormal')

# Encode the labels: normal as 0, abnormal as 1
mitbih_train['target'] = mitbih_train['target'].replace({'Normal': 0, 'abnormal': 1})
mitbih_test['target'] = mitbih_test['target'].replace({'Normal': 0, 'abnormal': 1})

# target value and meanings
# class_mapping = {
#     0: 'Normal',
#     1: 'Abnormal'
# }

#drop null value  
mitbih_train = mitbih_train.dropna(how='any')
mitbih_test = mitbih_test.dropna(how='any')




C:\Users\Isabell Gurstein\AppData\Local\Temp\ipykernel_22680\781607896.py:30: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mitbih_train['target'] = mitbih_train['target'].replace({'Normal': 0, 'abnormal': 1})
C:\Users\Isabell Gurstein\AppData\Local\Temp\ipykernel_22680\781607896.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mitbih_test['target'] = mitbih_test['target'].replace({'Normal': 0, 'abnormal': 1})


In [5]:
mitbih_train["target"].value_counts()

target
0    72471
1     8652
Name: count, dtype: int64

In [6]:
mitbih_test["target"].value_counts()

target
0    18118
1     2166
Name: count, dtype: int64

In [7]:
# split train test set before resampling

X_train = mitbih_train.drop(columns=['target'], inplace=False) # drop the target column
X_test = mitbih_test.drop(columns=['target'], inplace=False) # drop the target column

y_train = mitbih_train['target'] # only the target column
y_test = mitbih_test['target'] # only the target column


# Resampling the train data with Oversampling before scaling with MinMaxScaler
ros = RandomOverSampler(random_state=0)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# Scaling with MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)


In [8]:
# convert resampling rescaling data back to dataframe to concat
X_train_scaled_df =  pd.DataFrame(X_train_scaled, columns=[f'c_{i}' for i in range(X_train_scaled.shape[1])])
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=[f'c_{i}' for i in range(X_test_scaled.shape[1])])
y_train_resampled_df = pd.DataFrame(y_train_resampled, columns=['target'])
y_test_df = pd.DataFrame(y_test, columns=['target']) 


# concat X_train, y_train/ X_test, y_test
mitbih_train_clean_default = pd.concat(
    [
        X_train_scaled_df,
        y_train_resampled_df
    ], axis=1)

mitbih_test_clean_default = pd.concat(
    [  
        X_test_scaled_df,
        y_test_df.reset_index(drop=True)
    ], axis=1)


In [9]:
#save clean data to 
# mitbih_train_clean_minmax_oversampling.csv  
# mitbih_test_clean_minmax_oversampling.csv

mitbih_train_clean_default.to_csv(OutputFiles.get('train'), index=False)
mitbih_test_clean_default.to_csv(OutputFiles.get('test'), index=False)

In [10]:
from datetime import datetime
# Display the running time
print("Current time:", datetime.now())

Current time: 2024-11-13 10:06:11.105349
